In [ ]:
import sys
import os
import json

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))
from Raydium.adapter.RaydiumAdapter import RaydiumAdapter
from SaucerSwap.adapter.SaucerSwapAdapter import SaucerSwapAdapter

In [ ]:
# Config
CONFIG_PATH = "/root/Repositorios/LiquidityProvider/SaucerSwap/config/hedera.saucerswap.yaml"

# Parámetros de ejemplo: HBAR -> USDC (exact_in)
token_in = "HBAR"
token_out = "0.0.456858"
amount_out = 1_330_000
fee_bps = 1500
kind = "exact_out"
route_hops = []

adapter = SaucerSwapAdapter(CONFIG_PATH)
print("EVM address:", adapter.evm_address)

# 1) Quote
quote = adapter.get_quote(
    token_in=token_in,
    token_out=token_out,
    amount=amount_out,
    kind=kind,
    fee_bps=fee_bps,
    route_hops=route_hops
)
print("Quote:")
print(json.dumps(quote, indent=2))

# 2) Prepare (auto-asociación y auto-approve incluidos si son necesarios)
prep = adapter.swap_prepare(
    token_in=token_in,
    token_out=token_out,
    amount=amount_out,
    kind=kind,
    fee_bps=fee_bps,
    slippage_bps=50,
    deadline_s=300,
    route_hops=route_hops,
    recipient_evm=None   # usa la EVM address derivada del keyfile
)
print("\nSwap (prepare):")
print(json.dumps({
  "to": prep["to"],
  "value": prep["value"],
  "gasEstimate": prep["gasEstimate"],
  "data_prefix": prep["data"][:66] + "...",
  "association": prep.get("association"),
  "approve_present": prep.get("approve") is not None
}, indent=2))

# 3) Enviar (approve MAX_UINT256 si aplica + swap)
#    Espera a recibo por defecto (wait=True)
result = adapter.swap_send(prep, wait=True)
print("\nResultado envío:")
print(json.dumps(result, indent=2))

In [ ]:
def print_pretty_position(title, position):
    """
    Imprime de forma legible y clara la información de una posición.
    Si la posición existe, muestra los detalles relevantes.
    Si no existe, muestra el motivo o error.
    """
    print("=" * 60)
    print(f"{title}")
    print("=" * 60)
    if position.get("exists"):
        print("✅ Posición encontrada:")
        # Imprime los detalles de forma legible (indentado)
        print(json.dumps(position["details"], indent=2, ensure_ascii=False))
    else:
        print("❌ Posición NO encontrada.")
        detalles = position.get("details", {})
        if detalles:
            print("Detalles:")
            print(json.dumps(detalles, indent=2, ensure_ascii=False))
    print("\n")

# Consulta y muestra la posición en Raydium
# adapter = RaydiumAdapter("../Raydium/config/solana.raydium.yaml")
# res_raydium = adapter.check_position_exists_tool(
#     position_nft_mint="GBYTTW6jMYGZUN5p4sLCVR14LMtaJqLWpjzD79k7FyLw"
# )
# print_pretty_position("Raydium (Solana)", res_raydium)

# Consulta y muestra la posición en SaucerSwap
adapter = SaucerSwapAdapter("/root/Repositorios/LiquidityProvider/SaucerSwap/config/hedera.saucerswap.yaml")
res_saucer = adapter.check_position_exists_tool(serial=67538)
print_pretty_position("SaucerSwap (Hedera)", res_saucer)
